In [1]:
#Pretty print
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [2]:
# Picle
import pickle
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [3]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

C:\Users\miki_\Anaconda3\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
# Plotting
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [6]:
#Path to data
path_data=r"E:\FMI\Magistar\2 semestar\Preporcvasti sistemi (Reccomendation System)\Project\EmTagger"
data_name="\\all_preproc_no_hashtags.pickle"

with open(path_data+data_name, 'rb') as f:
    d = pickle.load(f, encoding='latin1')

In [7]:
data_full = [i['text'].split() for i in d]
n_train = 173007
n_test = 43252



length_data=n_train

data = data_full[0:length_data]

# Create Dictionary
id2word = corpora.Dictionary(data)

# Create Corpus
texts = data

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           chunksize=1024,
                                           passes=7,
                                           alpha='auto',
                                           per_word_topics=True)

pprint(lda_model.print_topics(5))

[(5,
  '0.110*"#" + 0.023*"video" + 0.014*"pictur" + 0.013*"171208" + 0.012*"music" '
  '+ 0.010*"made" + 0.009*"star" + 0.008*"left" + 0.008*"releas" + 0.007*"i"'),
 (1,
  '0.026*"come" + 0.018*"free" + 0.016*"i\'m" + 0.011*"not" + 0.011*"have" + '
  '0.011*"more" + 0.010*"alway" + 0.010*"readi" + 0.010*"listen" + '
  '0.010*"share"'),
 (4,
  '0.022*"today" + 0.021*"year" + 0.019*"good" + 0.018*"game" + 0.015*"back" + '
  '0.015*"first" + 0.015*"happi" + 0.012*"week" + 0.011*"tonight" + '
  '0.011*"end"'),
 (6,
  '0.077*"the" + 0.031*"get" + 0.022*"thi" + 0.019*"via" + 0.019*"make" + '
  '0.017*"We" + 0.016*"best" + 0.015*"need" + 0.014*"your" + 0.013*"how"'),
 (7,
  '0.057*"I" + 0.029*"you" + 0.024*"love" + 0.023*"like" + 0.022*"A" + '
  '0.019*"one" + 0.019*"look" + 0.019*"for" + 0.017*"see" + 0.016*"go"')]


In [10]:
#Save model for later use
path_saved_model = './tmp/model_lda1.atmodel'
lda_model.save(path_saved_model)
#model = gensim.models.ldamodel.LdaModel.load(path_saved_model)

doc_lda = lda_model[corpus]

In [11]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -9.337062909941073


In [12]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.2532007421974387


In [13]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\miki_\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.419315 -0.042478       1        1  16.443899
6     -0.003346  0.414219       2        1  14.341710
4     -0.044612 -0.053429       3        1  11.111541
9     -0.052637 -0.046371       4        1   9.728839
3     -0.055196 -0.045453       5        1   8.626498
2     -0.036808 -0.048977       6        1   8.397941
8     -0.057081 -0.043691       7        1   8.319425
0     -0.055970 -0.045120       8        1   8.137474
1     -0.055457 -0.043862       9        1   7.955595
5     -0.058207 -0.044836      10        1   6.937088, topic_info=      Category          Freq       Term         Total  loglift  logprob
term                                                                   
18     Default  13986.000000        the  13986.000000  30.0000  30.0000
182    Default   9638.000000          #   9638.000000  29.0000  29.0000
261    Default  11904.000000          I  11904.000000  28.0000  28.0000
210    Default   6931.000000        new   6931.000000  27.0000  27.0000
14     Default   5601.000000        get   5601.000000  26.0000  26.0000
106    Default   6064.000000        you   6064.000000  25.0000  25.0000
3      Default   4219.000000        day   4219.000000  24.0000  24.0000
258    Default   4112.000000       time   4112.000000  23.0000  23.0000
250    Default   4995.000000       love   4995.000000  22.0000  22.0000
215    Default   3599.000000      pleas   3599.000000  21.0000  21.0000
350    Default   4692.000000       like   4692.000000  20.0000  20.0000
346    Default   4647.000000          A   4647.000000  19.0000  19.0000
317    Default   3395.000000         RT   3395.000000  18.0000  18.0000
185    Default   3350.000000      thank   3350.000000  17.0000  17.0000
112    Default   3957.000000        thi   3957.000000  16.0000  16.0000
28     Default   4026.000000        one   4026.000000  15.0000  15.0000
806    Default   2802.000000       live   2802.000000  14.0000  14.0000
70     Default   3007.000000       2017   3007.000000  13.0000  13.0000
159    Default   3884.000000       look   3884.000000  12.0000  12.0000
87     Default   3878.000000        for   3878.000000  11.0000  11.0000
178    Default   2813.000000     follow   2813.000000  10.0000  10.0000
32     Default   3076.000000      today   3076.000000   9.0000   9.0000
425    Default   3464.000000        via   3464.000000   8.0000   8.0000
415    Default   3022.000000       year   3022.000000   7.0000   7.0000
94     Default   2818.000000      great   2818.000000   6.0000   6.0000
446    Default   2587.000000       come   2587.000000   5.0000   5.0000
41     Default   3367.000000       make   3367.000000   4.0000   4.0000
217    Default   3542.000000        see   3542.000000   3.0000   3.0000
287    Default   2484.000000      check   2484.000000   2.0000   2.0000
53     Default   3329.000000         go   3329.000000   1.0000   1.0000
...        ...           ...        ...           ...      ...      ...
182    Topic10   9638.047852          #   9638.884766   2.6682  -2.2102
244    Topic10   2008.550781      video   2009.387573   2.6679  -3.7785
1560   Topic10   1189.724243     pictur   1190.561035   2.6676  -4.3022
167    Topic10   1064.500244      music   1065.336914   2.6675  -4.4134
77495  Topic10   1144.926270     171208   1145.881226   2.6675  -4.3405
721    Topic10    875.312866       made    876.149658   2.6673  -4.6091
161    Topic10    812.948792       star    813.785522   2.6673  -4.6830
1282   Topic10    729.650696       left    730.487549   2.6671  -4.7911
160    Topic10    676.812439     releas    677.649170   2.6671  -4.8662
3347   Topic10    647.695496          i    648.532227   2.6670  -4.9102
61     Topic10    643.067383     offici    643.904114   2.6670  -4.9174
3536   Topic10    611.235901      guess    612.072632   2.6669  -4.9682
1976   Topic10    566.132935      place    566.969727   2.6668  -5.04

In [ ]:
lda_model.print_topics(20)

In [15]:
#n_test = 4048
n_validation = 0
temp_start = n_train+n_validation
d_test = d[temp_start: temp_start+n_test]

data_test = [(t["text"],t['hashtags']) for t in d_test]
data_testX = [ x[0].split() for x in data_test ]
data_testY = [ x[1] for x in data_test ]

id2wordT = corpora.Dictionary(data_testX)
textsT = data_testX
corpusT = [id2wordT.doc2bow(t) for t in textsT]

In [18]:
#res = format_topics_sentences(ldamodel=lda_model, corpus=corpusT, texts=data_testX)

In [19]:
def get_topic_for_hastag(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    result = []
    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        topic_num = sorted(row[0], key=lambda x: (x[1]), reverse=True)[0][0]
        result.append(topic_num)
    return(result)

In [21]:
res = get_topic_for_hastag(ldamodel=lda_model, corpus=corpusT, texts=data_testX)

In [22]:
wordsByTopic = {x:[x[0].lower() for x in lda_model.show_topic(x)] for x in range(10)}

In [23]:
def at_least_one(lst_one, lst_two):
    
    for x in lst_one:
        if x.lower() in lst_two:
            return 1
    return 0

In [55]:
result = []
for i,topic in enumerate(res):
    lstTweet = wordsByTopic[topic]
    lstHashtags = data_testYY[i]
    result.append(at_least_one(lstHashtags,lstTweet))
#print(result)


In [56]:
sum(result)

5226

In [57]:
data_testYY = [ list(set(a + [y for z in [re.findall('[A-Z][^A-Z]*', t) for t in a] for y in z])) for a in data_testY]

In [60]:
precison = sum(result)/n_test
precison

0.12082678257652825

In [61]:
F1 = 2*precison*precison/(precison+precison)
F1

0.12082678257652825